In [5]:
import re
from pathlib import Path

import pandas as pd


# ============================================
# 0. 경로 설정
# ============================================
BASE_DIR = Path(".")  # 이 스크립트가 있는 폴더 기준

# 입력 코호트 (ver45)
COHORT_PATH = BASE_DIR / "cohort" / "cohort_ver45_with_pci_rx_admin.csv"

# MIMIC-IV diagnoses_icd 경로 (필요에 따라 수정)
DIAGNOSES_PATH = (
    BASE_DIR
    / ".."
    / ".."
    / "data"
    / "MIMIC4-hosp-icu"
    / "diagnoses_icd.csv"
)

# 업로드한 SAS 기반 CCI 정의 파일
SAS_CCI_PATH = BASE_DIR / "ref" / "charlson_icd_map.txt"  # 파일 위치에 맞게 수정

# SAS를 파싱해 생성할 CSV
CHARLSON_MAP_CSV = BASE_DIR / "ref" / "charlson_icd_map.csv"

# 결과 코호트 (ver46)
OUTPUT_PATH = BASE_DIR / "cohort" / "cohort_ver46_with_cci.csv"


# ============================================
# 1. SAS 파일 → charlson_icd_map.csv 자동 생성
# ============================================

def build_charlson_map_from_sas(sas_path: Path, output_csv: Path):
    """
    업로드된 SAS CCI 코드 파일에서 ICD-10 매핑을 추출해
    charlson_icd_map.csv로 저장한다.
    """
    print(f"[Step 1] SAS CCI 파일 읽는 중: {sas_path}")

    # 인코딩 문제를 피하기 위해 cp949 + ignore 사용
    with open(sas_path, "r", encoding="cp949", errors="ignore") as f:
        sas_text = f.read()

    # SAS 변수명 기준 weight (agescore는 포함하지 않음)
    weight_map = {
        "AMI": 1,
        "CHF": 1,
        "PVD": 1,
        "CVD": 1,
        "DEMEN": 1,
        "PUL": 1,
        "CON": 1,
        "PEP": 1,
        "LIVER": 1,
        "DM": 1,
        "DM_COM": 2,
        "RENAL": 2,
        "HEMI": 2,
        "CANCER": 2,
        "META": 6,
        "SE_LIVER": 3,
        "AIDS": 6,
    }

    # SAS 변수명 → 사람이 보기 좋은 comorbidity 이름 매핑
    var_to_comorb = {
        "AMI": "mi",
        "CHF": "chf",
        "PVD": "pvd",
        "CVD": "cerebro",  # cerebrovascular disease
        "DEMEN": "dementia",
        "PUL": "copd",  # chronic pulmonary disease
        "CON": "rheumatic",  # connective tissue disease
        "PEP": "pud",  # peptic ulcer disease
        "LIVER": "mild_liver",
        "DM": "diabetes",
        "DM_COM": "diabetes_comp",
        "RENAL": "renal",
        "HEMI": "hemi_para",
        "CANCER": "any_malignancy",
        "META": "metastatic_solid_tumor",
        "SE_LIVER": "modsev_liver",
        "AIDS": "aids_hiv",
    }

    rows = []

    # 패턴 예시:
    # IF msick_cd2='I21' OR msick_cd2='I22' OR ... OR ssick_cd2='I252'
    # THEN AMI=1; else AMI=0;
    pattern = r"IF\s+(.*?)\s+then\s+(\w+)\s*=\s*1"

    matches = re.findall(pattern, sas_text, flags=re.IGNORECASE | re.DOTALL)
    print(f"[Step 1] IF ... THEN ...=1; 패턴 매칭 개수: {len(matches)}")

    for conds, varname in matches:
        var_upper = varname.upper()

        # 조건문 안의 모든 '코드' 추출
        codes = re.findall(r"'([A-Za-z0-9\.]+)'", conds)

        for code in codes:
            icd_prefix = code.replace(".", "").strip().upper()

            # comorbidity 이름과 weight 결정
            comorb = var_to_comorb.get(var_upper, var_upper.lower())
            weight = weight_map.get(var_upper, 1)

            rows.append(
                {
                    "icd_version": 10,  # 이 SAS는 ICD-10 기준이라고 가정
                    "icd_code_prefix": icd_prefix,
                    "comorbidity": comorb,
                    "weight": weight,
                }
            )

    df_map = pd.DataFrame(rows).drop_duplicates()

    output_csv.parent.mkdir(parents=True, exist_ok=True)
    df_map.to_csv(output_csv, index=False, encoding="utf-8")

    print(f"[Step 1] Charlson ICD map 저장 완료: {output_csv}")
    print(df_map.head())


# ============================================
# 2. diagnoses_icd + charlson_icd_map.csv → CCI 계산
# ============================================

def normalize_icd(code: str) -> str:
    """
    ICD 코드에서 공백 제거, 대문자 변환, '.' 제거.
    접두(prefix) 매칭용 정규화 함수.
    """
    if pd.isna(code):
        return ""
    s = str(code).strip().upper().replace(".", "")
    return s


def map_icd_to_charlson(dx_df: pd.DataFrame, map_df: pd.DataFrame) -> pd.DataFrame:
    """
    diagnoses_icd 서브셋(dx_df)에 대해 Charlson 매핑(map_df)을 적용해
    hadm_id - comorbidity - weight 테이블을 생성한다.

    dx_df: columns - hadm_id, icd_version, icd_code_norm
    map_df: columns - icd_version, icd_code_prefix_norm, comorbidity, weight
    """
    dx_work = dx_df[["hadm_id", "icd_version", "icd_code_norm"]].copy()
    results = []

    # prefix 길이 (길이가 긴 코드부터 우선)
    for plen in [5, 4, 3]:
        tmp = dx_work.copy()
        tmp["prefix"] = tmp["icd_code_norm"].str[:plen]

        map_sub = map_df.copy()
        map_sub["prefix"] = map_sub["icd_code_prefix_norm"].str[:plen]

        merged = tmp.merge(
            map_sub[["icd_version", "prefix", "comorbidity", "weight"]],
            on=["icd_version", "prefix"],
            how="inner",
        )

        merged = merged[["hadm_id", "comorbidity", "weight"]].drop_duplicates()
        results.append(merged)

    if not results:
        return pd.DataFrame(columns=["hadm_id", "comorbidity", "weight"])

    mapped = pd.concat(results, ignore_index=True).drop_duplicates()
    return mapped


def compute_cci_for_cohort():
    # 2-1. 코호트, diagnoses_icd, charlson map 로드
    print(f"[Step 2] Cohort 로드: {COHORT_PATH}")
    cohort = pd.read_csv(COHORT_PATH)
    print("  Cohort rows:", len(cohort))

    print(f"[Step 2] diagnoses_icd 로드: {DIAGNOSES_PATH}")
    dx = pd.read_csv(DIAGNOSES_PATH)
    print("  Diagnoses rows:", len(dx))

    print(f"[Step 2] Charlson map 로드: {CHARLSON_MAP_CSV}")
    charlson_map = pd.read_csv(CHARLSON_MAP_CSV)
    print("  Charlson map rows:", len(charlson_map))

    # 2-2. 코호트 hadm_id에 해당하는 진단만 사용
    hadm_ids = cohort["hadm_id"].unique()
    dx_sub = dx[dx["hadm_id"].isin(hadm_ids)].copy()
    print("  Diagnoses rows (cohort hadm):", len(dx_sub))

    # 2-3. ICD 코드 정규화
    dx_sub["icd_code_norm"] = dx_sub["icd_code"].apply(normalize_icd)
    charlson_map["icd_code_prefix_norm"] = (
        charlson_map["icd_code_prefix"]
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(".", "", regex=False)
    )

    # 2-4. ICD → Charlson 매핑
    dx_charlson = map_icd_to_charlson(dx_sub, charlson_map)
    print("  Mapped Charlson rows:", len(dx_charlson))

    # 2-5. hadm_id × comorbidity flag (0/1)
    if dx_charlson.empty:
        # 매핑이 하나도 안 된 경우
        cci_flags = pd.DataFrame({"hadm_id": hadm_ids})
        cci_flags["cci_score"] = 0
        cci_flags["cci_category"] = "0"
    else:
        cci_flags = (
            dx_charlson.assign(flag=1)
            .pivot_table(
                index="hadm_id",
                columns="comorbidity",
                values="flag",
                aggfunc="max",
                fill_value=0,
            )
            .reset_index()
        )
        print("  CCI flag table shape:", cci_flags.shape)

        # comorbidity별 weight 테이블
        weight_table = (
            charlson_map[["comorbidity", "weight"]]
            .drop_duplicates()
            .set_index("comorbidity")["weight"]
        )

        def compute_cci_score_row(row: pd.Series) -> int:
            score = 0
            for comorbidity, w in weight_table.items():
                if comorbidity in row and row[comorbidity] == 1:
                    score += int(w)
            return score

        cci_flags["cci_score"] = cci_flags.apply(compute_cci_score_row, axis=1)

        # CCI 범주화 (0 / 1–2 / ≥3)
        def categorize_cci(score: float) -> str:
            if pd.isna(score):
                return "0"
            score = int(score)
            if score == 0:
                return "0"
            elif 1 <= score <= 2:
                return "1_2"
            else:
                return "3_plus"

        cci_flags["cci_category"] = cci_flags["cci_score"].apply(categorize_cci)

    print("[Step 2] CCI 예시:")
    print(cci_flags[["hadm_id", "cci_score", "cci_category"]].head())

    # 2-6. 코호트와 merge
    cohort_cci = cohort.merge(
        cci_flags[["hadm_id", "cci_score", "cci_category"]],
        on="hadm_id",
        how="left",
    )

    # 매핑 안 된 경우 NaN → 0, "0"
    cohort_cci["cci_score"] = cohort_cci["cci_score"].fillna(0).astype(int)
    cohort_cci["cci_category"] = cohort_cci["cci_category"].fillna("0")

    print("[Step 2] 최종 코호트 shape:", cohort_cci.shape)
    print(cohort_cci[["hadm_id", "cci_score", "cci_category"]].head())

    # 2-7. 저장
    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    cohort_cci.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")
    print(f"[Step 2] 저장 완료: {OUTPUT_PATH}")


# ============================================
# main
# ============================================
if __name__ == "__main__":
    # 1) SAS 파일 → CSV 변환
    build_charlson_map_from_sas(SAS_CCI_PATH, CHARLSON_MAP_CSV)

    # 2) CCI 계산하여 cohort ver46 생성
    compute_cci_for_cohort()


[Step 1] SAS CCI 파일 읽는 중: ref\charlson_icd_map.txt
[Step 1] IF ... THEN ...=1; 패턴 매칭 개수: 5
[Step 1] Charlson ICD map 저장 완료: ref\charlson_icd_map.csv
   icd_version icd_code_prefix             comorbidity  weight
0           10             I21                      mi       1
1           10             I22                      mi       1
2           10            I252                      mi       1
6           10             C77  metastatic_solid_tumor       6
7           10             C78  metastatic_solid_tumor       6
[Step 2] Cohort 로드: cohort\cohort_ver45_with_pci_rx_admin.csv
  Cohort rows: 1930
[Step 2] diagnoses_icd 로드: ..\..\data\MIMIC4-hosp-icu\diagnoses_icd.csv
  Diagnoses rows: 6364488
[Step 2] Charlson map 로드: ref\charlson_icd_map.csv
  Charlson map rows: 22
  Diagnoses rows (cohort hadm): 27892
  Mapped Charlson rows: 618
  CCI flag table shape: (552, 5)
[Step 2] CCI 예시:
comorbidity   hadm_id  cci_score cci_category
0            20071347          1          1_2
1         